<a href="https://colab.research.google.com/github/Herudaio/data-playground-blogpost/blob/master/notebooks/data_playground_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install folium >> /dev/null 2>&1
!pip install folium --upgrade  >> /dev/null 2>&1
!pip install geopandas >> /dev/null 2>&1

In [0]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.affinity import scale

greater_london_bounds = -0.508813, 51.286911, 0.335677, 51.69207
lon_min, lat_min, lon_max, lat_max = greater_london_bounds
lon_grid_size, lat_grid_size = 58, 45

lon_range = lon_max - lon_min
d_lon = lon_range / lon_grid_size
lat_range = lat_max - lat_min
d_lat = lat_range / lat_grid_size

grid_coordinates = [[n_lon, n_lat] 
                    for n_lon in range(lon_grid_size) 
                    for n_lat in range(lat_grid_size)]
df = pd.DataFrame(grid_coordinates, columns=["n_lon", "n_lat"])
df["lon"] = df["n_lon"].apply(lambda n: n * d_lon + lon_min)
df["lat"] = df["n_lat"].apply(lambda n: n * d_lat + lat_min)


def rect_from_coords(lon, lat):
  return Polygon([(lon - d_lon/2, lat - d_lat/2), 
                  (lon - d_lon/2, lat + d_lat/2),
                  (lon + d_lon/2, lat + d_lat/2),
                  (lon + d_lon/2, lat - d_lat/2)])

df["geometry"] = df[["lon", "lat"]].apply(lambda row: 
                                          rect_from_coords(row["lon"], 
                                                           row["lat"]), 
                                          axis=1)
df["center"] = df["geometry"].apply(lambda shape: shape.centroid)

gdf = gpd.GeoDataFrame(df, geometry="geometry")
gdf.crs = {'init': 'EPSG:4326'}
gdf["center_local"] = gdf.to_crs(epsg=27700)["geometry"].apply(lambda shape: 
                                                               shape.centroid)

In [0]:
from pyproj import Proj, transform

worldProj = Proj(init='epsg:4326')
localProj = Proj(init='epsg:27700')


def convert_world_coords_to_local(lon, lat):
  return transform(worldProj, localProj, lon, lat)

tower_of_london_coords_local = convert_world_coords_to_local(0.0759, 51.5081)
tower_of_london_local = Point(tower_of_london_coords_local)

gdf["tower_of_london_dist"] = gdf["center_local"].apply(
    lambda point: point.distance(tower_of_london_local))

In [0]:
from shapely.ops import cascaded_union

x_min, y_min = convert_world_coords_to_local(lon_min, lat_min)
x_max, y_max = convert_world_coords_to_local(lon_max, lat_max)
x_range = x_max - x_min
y_range = y_max - y_min

restaurants = [Point(x * x_range + x_min, y * y_range + y_min)
               for x, y in np.random.rand(1000,2)]

restaurants_union = cascaded_union(restaurants)

gdf["closest_restaurant_dist"] = gdf["center_local"].apply(
    lambda point: point.distance(restaurants_union))

In [0]:
def restaurants_within_1_km(point):
  return len([1 for restaurant in restaurants if point.distance(restaurant) < 1000])


gdf["restaurants_within_1km"] = gdf["center_local"].apply(
    lambda point: restaurants_within_1_km(point))

In [6]:
from matplotlib import cm
import matplotlib.colors
import folium
import branca.colormap as bcm

feature_name = "tower_of_london_dist"
# feature_name = "closest_restaurant_dist"
# feature_name = "restaurants_within_1km"

data = gdf[["geometry", feature_name]]

folium_map = folium.Map(location=[51.5074, -0.1278], 
                        zoom_start=10, 
                        tiles='cartodbpositron')

vmin = gdf[feature_name].min()
vmax = gdf[feature_name].max()
cmap = bcm.linear.Paired_06.scale(vmin, vmax)


def style_function(feature):
  return {
      'fillColor': cmap(feature["properties"][feature_name]),
      'fillOpacity': 0.4,
      'color':'white',
      'weight':0.7
  }

folium.GeoJson(data, style_function=style_function).add_to(folium_map)

norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
mapper = cm.ScalarMappable(norm=norm, cmap=cm.jet)

colors = mapper.to_rgba(np.linspace(vmin, vmax, 10)).tolist()
colormap = bcm.LinearColormap(colors, vmin=vmin, vmax=vmax)
colormap.caption = feature_name
folium_map.add_child(colormap)

folium_map